Note: Get the data from the github repo: https://github.com/Georgepu1/cs263-final-project/

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/LSTM')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install torchtext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Note first get the data from the github to start data prep phase
import pandas as pd

sst_tr_dataset = pd.read_csv('sst_train_data.csv')
sst_val_dataset = pd.read_csv('sst_val_data.csv')
sst_test_dataset = pd.read_csv('sst_test_data.csv')

cola_tr_dataset = pd.read_csv('cola_train_data.csv')
cola_val_dataset = pd.read_csv('cola_val_data.csv')
cola_test_dataset = pd.read_csv('cola_test_data.csv')

print(sst_tr_dataset.size)
print(cola_tr_dataset.size)

concat_df = pd.concat([sst_tr_dataset,cola_tr_dataset]).sort_index()
concat_df.head()

15390
15390


,sentences,labels
0,The Rock is destined to be the 21st Century 's...,1
0,"Our friends won't buy this analysis, let alone...",1
1,The gorgeously elaborate continuation of `` Th...,1
1,One more pseudo generalization and I'm giving up.,1
2,Singer\/composer Bryan Adams contributes a sle...,1


In [ ]:
#vocabulary = []
#for idx, row in concat_df.iterrows():
#    for word in row["sentences"].split():
#      if word not in vocabulary:
#        vocabulary.append(word)

#print(len(vocabulary))

In [ ]:
#print(vocabulary)

# New Section

In [ ]:
from torchtext.vocab import GloVe, vocab
from torchtext.data.utils import get_tokenizer

global_vectors = GloVe(name='6B', dim=300)
tokenizer = get_tokenizer("basic_english")

.vector_cache/glove.6B.zip: 862MB [02:40, 5.37MB/s]                           
100%|█████████▉| 399999/400000 [00:48<00:00, 8288.33it/s]


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import random
import transformers
# from transformers import pipeline, AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoConfig


glove_vocab = vocab(global_vectors.stoi)
unk_token = "<unk>"
unk_index = 0
glove_vocab.insert_token("<unk>", unk_index)
glove_vocab.set_default_index(unk_index)
pretrained_embeddings = global_vectors.vectors
pretrained_embeddings = torch.cat((torch.zeros(1,pretrained_embeddings.shape[1]),pretrained_embeddings))

In [ ]:
type(global_vectors.stoi)

dict

In [ ]:
pretrained_embeddings

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
        [-0.2554, -0.2572,  0.1317,  ..., -0.2329, -0.1223,  0.3550],
        ...,
        [ 0.0757, -0.0405,  0.1834,  ...,  0.2184,  0.3097,  0.4376],
        [ 0.8145, -0.3622,  0.3119,  ...,  0.0755,  0.2841, -0.1756],
        [ 0.4292, -0.2969,  0.1501,  ...,  0.2898,  0.3262, -0.0591]])

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2022-06-12 02:00:48--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-06-12 02:00:48--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-06-12 02:00:48--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
print('Indexing word vectors.')

embeddings_index = {}
f = open('glove.6B.300d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [ ]:
#embeddings_index["the"]
len(embeddings_index.keys())

400000

In [ ]:
# def tokenize_sentences(x, tokenizer, max_words = 48):
#   x = tokenizer(x)
#   if len(x) < max_words:
#     x = x + [""] * (max_words - len(x))
#   else:
#     x = x[:max_words]

#   return glove_vocab(x)

# sst_tr_dataset['sentences'] = sst_tr_dataset['sentences'].apply(lambda x: tokenize_sentences(x, tokenizer))
# sst_val_dataset['sentences'] = sst_val_dataset['sentences'].apply(lambda x: tokenize_sentences(x, tokenizer))
# sst_test_dataset['sentences'] = sst_test_dataset['sentences'].apply(lambda x: tokenize_sentences(x, tokenizer))
# cola_tr_dataset['sentences'] = cola_tr_dataset['sentences'].apply(lambda x: tokenize_sentences(x, tokenizer))
# cola_val_dataset['sentences'] = cola_val_dataset['sentences'].apply(lambda x: tokenize_sentences(x, tokenizer))
# cola_test_dataset['sentences'] = cola_test_dataset['sentences'].apply(lambda x: tokenize_sentences(x, tokenizer))


In [ ]:
from collections import namedtuple

gen_batch_fields = ['sst_input_text', 'sst_input_inds', 'sst_target', 'cola_input_text', 'cola_input_inds', 'cola_target']
GenBatch = namedtuple('GenBatch', field_names=gen_batch_fields, defaults=[None] * len(gen_batch_fields))

In [ ]:
# Dataset for LSTM + Embedding / BERT
class MultitaskDataset(torch.utils.data.Dataset):
  def __init__(self, vocab, tokenizer, sst_X, sst_y, cola_X, cola_y, max_len=10000):
    assert len(sst_X) == len(sst_y), "SST data not one to one"
    assert len(cola_X) == len(cola_y), "CoLA data not one to one"

    self.max_len = max_len
    self.tokenizer = tokenizer
    self.vocab = vocab
    self.data = []
    min_samples = min(len(sst_X), len(cola_X))
    sst_X = sst_X[:min(min_samples, max_len)]
    sst_y = sst_y[:min(min_samples, max_len)]
    cola_X = cola_X[:min(min_samples, max_len)]
    cola_y = cola_y[:min(min_samples, max_len)]
    self.load_data(sst_X, sst_y, cola_X, cola_y)

  def load_data(self, sst_X, sst_y, cola_X, cola_y):
    for (cur_sst_X, cur_sst_y, cur_cola_X, cur_cola_y) in zip(sst_X, sst_y, cola_X, cola_y):
      self.data.append({
          'sst_input': cur_sst_X, 
          'sst_target': cur_sst_y, 
          'cola_input': cur_cola_X, 
          'cola_target': cur_cola_y,
      })

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
      # Get element consisting of sst_X, sst_y, cola_X, and cola_y
    return self.data[index]

  def tokenize_sentences(self, x, max_words=32):
    x = self.tokenizer(x)
    if len(x) < max_words:
      x = x + [""] * (max_words - len(x))
    else:
      x = x[:max_words]
    return self.vocab(x)
    #return [self.vocab[xi] for xi in x]
    
  def collate_fn(self, batch):
    sst_input_text = [x['sst_input'] for x in batch]
    sst_inputs = [self.tokenize_sentences(x['sst_input']) for x in batch]
    cola_input_text = [x['cola_input'] for x in batch]
    cola_inputs = [self.tokenize_sentences(x['cola_input']) for x in batch]

    return GenBatch(
      sst_input_text=sst_input_text,
      sst_input_inds=sst_inputs,
      sst_target=[x['sst_target'] for x in batch],
      cola_input_text=cola_input_text,
      cola_input_inds=cola_inputs,
      cola_target=[x['cola_target'] for x in batch]
    )

In [ ]:
train_set = MultitaskDataset(glove_vocab, tokenizer, sst_tr_dataset['sentences'], sst_tr_dataset['labels'], 
                             cola_tr_dataset['sentences'], cola_tr_dataset['labels'])
val_set = MultitaskDataset(glove_vocab, tokenizer, sst_val_dataset['sentences'], sst_val_dataset['labels'], 
                           cola_val_dataset['sentences'], cola_val_dataset['labels'])
test_set = MultitaskDataset(glove_vocab, tokenizer, sst_test_dataset['sentences'], sst_test_dataset['labels'], 
                             cola_test_dataset['sentences'], cola_test_dataset['labels'])

In [ ]:
train_loader = DataLoader(train_set, shuffle=True, batch_size=8,
                          collate_fn=train_set.collate_fn)
val_loader  = DataLoader(val_set, shuffle=False, batch_size=8,
                          collate_fn=val_set.collate_fn)
test_loader  = DataLoader(test_set, shuffle=False, batch_size=8,
                          collate_fn=test_set.collate_fn)

In [ ]:
len(global_vectors.stoi.keys())

400000

In [ ]:
matrix_len = len(global_vectors.stoi.keys())
weights_matrix = np.zeros((matrix_len, 300))
words_found = 0

for i, word in enumerate(global_vectors.stoi.keys()):
    try: 
        weights_matrix[i] = embeddings_index[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(300, ))

In [ ]:
len(weights_matrix[1])
weights_matrix = torch.from_numpy(weights_matrix)

In [ ]:
weights_matrix.size()

torch.Size([400000, 300])

In [ ]:
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.size()
    #num_embeddings=len(weights_matrix)
    #embedding_dim=len(weights_matrix[1])
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

In [ ]:
num_embeddings, embedding_dim = weights_matrix.size()
emb_layer = nn.Embedding(num_embeddings, embedding_dim)
print(num_embeddings, embedding_dim)

400000 300


In [ ]:
a = torch.randint(22434,(8,32))
#a.size()


a[0][0]=22433


emb_layer(a)

tensor([[[ 1.3227e+00, -1.4992e+00,  3.7658e-01,  ...,  8.3209e-01,
           4.7416e-01, -1.8094e-01],
         [-4.4425e-01, -3.7590e-01,  4.4801e-01,  ...,  2.7711e-01,
          -1.7417e+00, -1.3152e+00],
         [ 5.2720e-02,  4.3240e-01, -1.4621e-01,  ..., -1.7341e-01,
          -5.8025e-01, -1.5264e-01],
         ...,
         [ 7.9086e-01,  4.7315e-01,  7.8253e-01,  ..., -1.1050e+00,
          -4.8477e-01,  4.3902e-01],
         [-7.4464e-01, -1.5717e-01,  6.4815e-02,  ..., -2.5423e-01,
          -6.3142e-01,  5.3278e-01],
         [ 1.0733e+00, -1.7190e+00, -1.3356e-01,  ...,  1.4763e+00,
          -4.5795e-01, -6.4814e-01]],

        [[ 2.3314e-01, -5.6579e-01, -2.3204e-01,  ..., -1.6839e+00,
           2.2549e+00, -1.2142e+00],
         [-2.2065e+00, -3.5880e-01,  3.7841e-01,  ..., -7.3157e-01,
           5.7307e-01,  4.4282e-02],
         [-2.0704e+00,  8.1091e-03, -1.2591e-01,  ...,  5.0151e-01,
           1.6859e+00,  1.4925e+00],
         ...,
         [ 4.5795e-01, -9

In [ ]:
import torch.nn.functional as F

class MultiTaskLSTM(torch.nn.Module):
  def __init__(self, weights_matrix, emb_dim, hidden_dim):
    super().__init__()
    #self.embeddings = nn.EmbeddingBag.from_pretrained(pretrain_emb)
    self.embeddings, num_embeddings, emb_dim = create_emb_layer(weights_matrix, True)
    self.lstm = nn.LSTM(emb_dim, hidden_dim, batch_first=True, bidirectional=True)
    self.linears = nn.ModuleList([nn.Linear(hidden_dim, 2) for i in range(2)])
    self.dropout = nn.Dropout(0.2)
    self.init_weights()

  def init_weights(self):
    initrange = 0.5
    self.linears[0].weight.data.uniform_(-initrange, initrange)
    self.linears[1].weight.data.uniform_(-initrange, initrange)
    self.linears[0].bias.data.zero_()
    self.linears[1].bias.data.zero_()

  def forward(self, x, ind):
    #print('inside forward')
    #print(x.shape)
    x = self.embeddings(x)
    #print(x.shape)
    x = self.dropout(x)
    #print(x.shape)
    #print('done with dropout')
    # x_pack = pack_padded_sequence(x, s, batch_first=True, enforce_sorted=False)
    lstm_out, (ht, ct) = self.lstm(x)
    logits = self.linears[ind](ht[-1])

    return F.softmax(logits, dim=-1)

print(weights_matrix)
model = MultiTaskLSTM(weights_matrix, 300, 64)

tensor([[ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
        [-0.2554, -0.2572,  0.1317,  ..., -0.2329, -0.1223,  0.3550],
        [-0.1256,  0.0136,  0.1031,  ..., -0.3422, -0.0224,  0.1368],
        ...,
        [ 0.0757, -0.0405,  0.1834,  ...,  0.2184,  0.3097,  0.4376],
        [ 0.8145, -0.3622,  0.3119,  ...,  0.0755,  0.2841, -0.1756],
        [ 0.4292, -0.2969,  0.1501,  ...,  0.2898,  0.3262, -0.0591]],
       dtype=torch.float64)


In [ ]:
#param_groups = [{'params': model.parameters(), 'lr': 0.00001, 'weight_decay': 0.01}]
#optimizer = AdamW(params=param_groups

In [ ]:
learning_rate = 1e-5
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
# TODO: maybe need to truncate and clip gradient
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

def eval_metrics(model, dl, criterion):
  model.eval()
  sst_acc = 0
  sst_loss = 0
  cola_acc = 0
  cola_loss = 0
  total = 0
  for sample in dl:
    model_sst_inp = torch.tensor(sample.sst_input_inds)
    model_cola_inp = torch.tensor(sample.cola_input_inds)
    y1_pred = model(model_sst_inp, 0).reshape(-1, 2)
    y2_pred = model(model_cola_inp, 1).reshape(-1, 2)
    sample_sst_target = torch.tensor(sample.sst_target).type(torch.long).reshape(-1)
    sample_cola_target = torch.tensor(sample.cola_target).type(torch.long).reshape(-1)

    sst_loss = criterion(y1_pred, sample_sst_target)
    cola_loss = criterion(y2_pred, sample_cola_target)
    
    # print('Length: {}'.format(y1_pred.shape[0]))
    sst_acc += (torch.eq(sample_sst_target,y1_pred.argmax(1))).sum().item()
    cola_acc += (torch.eq(sample_cola_target, y2_pred.argmax(1))).sum().item()
    sst_loss += sst_loss.item()
    cola_loss += cola_loss.item()
    total += y1_pred.shape[0]

  return sst_acc/total, sst_loss/total, cola_acc/total, cola_loss/total

In [ ]:
from tqdm import tqdm
epochs = 10

# offset input is required due to how embeddings are loaded in
#offset_input = torch.tensor([0])

for _ in range(epochs):
  
  tr_count = 0
  sst_tr_acc = 0
  sst_tr_loss = 0
  cola_tr_acc = 0
  cola_tr_loss = 0
  for sample in tqdm(train_loader):
    # Note can also set the data to a decide (cuda)
    model.zero_grad()
    # print(type(sample.sst_input_inds))
    # print(torch.tensor(sample.sst_input_inds))
    # print(torch.tensor(sample.sst_input_inds).type(torch.long))
    model_sst_inp = torch.tensor(sample.sst_input_inds)
    #print(model_sst_inp)
    model_cola_inp = torch.tensor(sample.cola_input_inds)
    #print(model_cola_inp.shape)
    y1_pred = model(model_sst_inp, 0).reshape(-1, 2)
    y2_pred = model(model_cola_inp, 1).reshape(-1, 2)
    sample_sst_target = torch.tensor(sample.sst_target).type(torch.long).reshape(-1)
    sample_cola_target = torch.tensor(sample.cola_target).type(torch.long).reshape(-1)
    #print(y1_pred.shape)
    #print(sample_sst_target.shape)
    sst_loss = criterion(y1_pred, sample_sst_target)
    cola_loss = criterion(y2_pred, sample_cola_target)
    # multitask loss
    loss = sst_loss + cola_loss
    loss.backward()

    # intermediate variabels stores embedding of x and computes
    # m_output.grad w.r.t. this and calculate the MSE of the m_output.grad(emb_x)
    # norm and 1.0 and use relu on; before you do the square, pass it through a relu
    # so everything les than 1.0 wont be counted to the square.
    # MSE (m_output.grad(x), 1.0) calulate gradient of M output w.r.t. x's embedding space
    # To avoid overfitting, calculating regularization term can use a varied version of x
    # instead of the original (e.g. add gaussian noise around embeddings of x); can
    # also minimize discrepancy on the two for robustness of model
    # Lipschitz-regularized loss
    optimizer.step()
    sst_tr_acc += (torch.eq(sample_sst_target,y1_pred.argmax(1))).sum().item()
    cola_tr_acc += (torch.eq(sample_cola_target, y2_pred.argmax(1))).sum().item()
    sst_tr_loss += sst_loss.item()
    cola_tr_loss += cola_loss.item()
    tr_count += y1_pred.shape[0]
    
  print("SST Train accuracy: {}, CoLA Train accuracy: {}, SST Train loss: {}, CoLA Train loss: {}".format(sst_tr_acc/tr_count, cola_tr_acc/tr_count, sst_tr_loss/tr_count, cola_tr_loss/tr_count))
  val_sst_acc, val_sst_loss, val_cola_acc, val_cola_loss = eval_metrics(model, val_loader, criterion)
  print("SST Val accuracy: {}, CoLA Val accuracy: {}, SST Val loss: {}, CoLA Val loss: {} ".format(val_sst_acc, val_cola_acc, val_sst_loss, val_cola_loss))

100%|██████████| 962/962 [00:45<00:00, 21.19it/s]


SST Train accuracy: 0.49889538661468485, CoLA Train accuracy: 0.6276803118908382, SST Train loss: 0.08720184368619367, CoLA Train loss: 0.08275734302360876
SST Val accuracy: 0.530373831775701, CoLA Val accuracy: 0.7149532710280374, SST Val loss: 0.0015780782559886575, CoLA Val loss: 0.0015586630906909704 


100%|██████████| 962/962 [00:47<00:00, 20.38it/s]


SST Train accuracy: 0.5494476933073424, CoLA Train accuracy: 0.7021442495126705, SST Train loss: 0.085783430993983, CoLA Train loss: 0.07742427472095353
SST Val accuracy: 0.5899532710280374, CoLA Val accuracy: 0.719626168224299, SST Val loss: 0.0016858924645930529, CoLA Val loss: 0.0015598353929817677 


100%|██████████| 962/962 [00:46<00:00, 20.82it/s]


...

SST Train accuracy: 0.6337881741390513, CoLA Train accuracy: 0.7026640675763483, SST Train loss: 0.08161928926372466, CoLA Train loss: 0.07617235453660183
SST Val accuracy: 0.6647196261682243, CoLA Val accuracy: 0.719626168224299, SST Val loss: 0.0016607041470706463, CoLA Val loss: 0.0016068343538790941 


100%|██████████| 962/962 [00:45<00:00, 21.00it/s]


SST Train accuracy: 0.6471734892787524, CoLA Train accuracy: 0.7026640675763483, SST Train loss: 0.08057181860736007, CoLA Train loss: 0.07615736414272341
SST Val accuracy: 0.6810747663551402, CoLA Val accuracy: 0.719626168224299, SST Val loss: 0.0016491982387378812, CoLA Val loss: 0.001610399573110044 


100%|██████████| 962/962 [00:47<00:00, 20.37it/s]


SST Train accuracy: 0.6618583495776478, CoLA Train accuracy: 0.7026640675763483, SST Train loss: 0.07926615351899403, CoLA Train loss: 0.07614195687651557
SST Val accuracy: 0.6904205607476636, CoLA Val accuracy: 0.719626168224299, SST Val loss: 0.0016052982537075877, CoLA Val loss: 0.0016141962260007858 


In [ ]:
test_sst_acc, test_sst_loss, test_cola_acc, test_cola_loss = eval_metrics(model, test_loader, criterion)
print("SST Test accuracy: {}, CoLA Val accuracy: {}, SST Val loss: {}, CoLA Val loss: {} ".format(test_sst_acc, test_cola_acc, test_sst_loss, test_cola_loss))

SST Test accuracy: 0.7115749525616698, CoLA Val accuracy: 0.6925996204933587, SST Val loss: 0.0020207613706588745, CoLA Val loss: 0.0032476233318448067 


### TODO: Adversarial attacks and Robustness Evaluation

In [ ]:
def attack():
  pass

In [ ]:
import pandas as pd

sst_adv_dataset = pd.read_csv('adversial_sst.csv')
cola_adv_dataset = pd.read_csv('adversial_cola.csv')

In [ ]:
adv_set = MultitaskDataset(glove_vocab, tokenizer, sst_adv_dataset['sentences'], sst_adv_dataset['labels'], cola_adv_dataset['sentences'], cola_adv_dataset['labels'])
adv_loader=DataLoader(adv_set, shuffle=False, batch_size=8, collate_fn=adv_set.collate_fn)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
adv_sst_acc, adv_sst_loss, adv_cola_acc, adv_cola_loss = eval_metrics(model, adv_loader, criterion)
print("SST ADV accuracy: {}, CoLA ADV accuracy: {}, SST ADV loss: {}, CoLA ADV loss: {} ".format(test_sst_acc, test_cola_acc, test_sst_loss, test_cola_loss))

SST ADV accuracy: 0.6425233644859814, CoLA ADV accuracy: 0.719626168224299, SST ADV loss: 0.0016325060278177261, CoLA ADV loss: 0.0015612159622833133 


In [ ]:
#BERT transfer attacks

sst_adv_dataset = pd.read_csv('bert_adversarial_sst.csv')
cola_adv_dataset = pd.read_csv('bert_adversarial_cola.csv')

adv_set = MultitaskDataset(glove_vocab, tokenizer, sst_adv_dataset['sentences'], sst_adv_dataset['labels'], cola_adv_dataset['sentences'], cola_adv_dataset['labels'])
adv_loader  = DataLoader(adv_set, shuffle=False, batch_size=8, collate_fn=adv_set.collate_fn)


In [ ]:
adv_sst_acc, adv_sst_loss, adv_cola_acc, adv_cola_loss = eval_metrics(model, adv_loader, criterion)
print("SST ADV accuracy: {}, CoLA ADV accuracy: {}, SST ADV loss: {}, CoLA ADV loss: {} ".format(adv_sst_acc, adv_cola_acc, adv_sst_loss, adv_cola_loss))

SST ADV accuracy: 0.5847457627118644, CoLA ADV accuracy: 0.8491525423728814, SST ADV loss: 0.002424434293061495, CoLA ADV loss: 0.001985773677006364 


In [ ]:
#ROBERTA transfer attack

sst_adv_dataset = pd.read_csv('roberta_adversarial_sst.csv')
cola_adv_dataset = pd.read_csv('roberta_adversarial_cola.csv')

adv_set = MultitaskDataset(glove_vocab, tokenizer, sst_adv_dataset['sentences'], sst_adv_dataset['labels'], 
                             cola_adv_dataset['sentences'], cola_adv_dataset['labels'])
adv_loader  = DataLoader(adv_set, shuffle=False, batch_size=8,
                          collate_fn=adv_set.collate_fn)

In [ ]:
adv_sst_acc, adv_sst_loss, adv_cola_acc, adv_cola_loss = eval_metrics(model, adv_loader, criterion)
print("SST ADV accuracy: {}, CoLA ADV accuracy: {}, SST ADV loss: {}, CoLA ADV loss: {} ".format(adv_sst_acc, adv_cola_acc, adv_sst_loss, adv_cola_loss))

SST ADV accuracy: 0.5428571428571428, CoLA ADV accuracy: 0.8767857142857143, SST ADV loss: 0.0023840838111937046, CoLA ADV loss: 0.002176337642595172 


In [ ]:
#Data aug
sst_adv_dataset = pd.read_csv('augmented_sst.csv')
cola_adv_dataset = pd.read_csv('augmented_cola.csv')

adv_set = MultitaskDataset(glove_vocab, tokenizer, sst_adv_dataset['sentences'], sst_adv_dataset['labels'], 
                             cola_adv_dataset['sentences'], cola_adv_dataset['labels'])
adv_loader  = DataLoader(adv_set, shuffle=False, batch_size=8,
                          collate_fn=adv_set.collate_fn)


In [ ]:
from tqdm import tqdm
epochs = 10

# offset input is required due to how embeddings are loaded in
#offset_input = torch.tensor([0])

for _ in range(epochs):
  
  tr_count = 0
  sst_tr_acc = 0
  sst_tr_loss = 0
  cola_tr_acc = 0
  cola_tr_loss = 0
  for sample in tqdm(adv_loader):
    # Note can also set the data to a decide (cuda)
    model.zero_grad()

    model_sst_inp = torch.tensor(sample.sst_input_inds)
    model_cola_inp = torch.tensor(sample.cola_input_inds)

    y1_pred = model(model_sst_inp, 0).reshape(-1, 2)
    y2_pred = model(model_cola_inp, 1).reshape(-1, 2)
    sample_sst_target = torch.tensor(sample.sst_target).type(torch.long).reshape(-1)
    sample_cola_target = torch.tensor(sample.cola_target).type(torch.long).reshape(-1)

    sst_loss = criterion(y1_pred, sample_sst_target)
    cola_loss = criterion(y2_pred, sample_cola_target)
    # multitask loss
    loss = sst_loss + cola_loss
    loss.backward()

    # intermediate variabels stores embedding of x and computes
    # m_output.grad w.r.t. this and calculate the MSE of the m_output.grad(emb_x)
    # norm and 1.0 and use relu on; before you do the square, pass it through a relu
    # so everything les than 1.0 wont be counted to the square.
    # MSE (m_output.grad(x), 1.0) calulate gradient of M output w.r.t. x's embedding space
    # To avoid overfitting, calculating regularization term can use a varied version of x
    # instead of the original (e.g. add gaussian noise around embeddings of x); can
    # also minimize discrepancy on the two for robustness of model
    # Lipschitz-regularized loss
    optimizer.step()
    sst_tr_acc += (torch.eq(sample_sst_target,y1_pred.argmax(1))).sum().item()
    cola_tr_acc += (torch.eq(sample_cola_target, y2_pred.argmax(1))).sum().item()
    sst_tr_loss += sst_loss.item()
    cola_tr_loss += cola_loss.item()
    tr_count += y1_pred.shape[0]
    
  print("SST Train accuracy: {}, CoLA Train accuracy: {}, SST Train loss: {}, CoLA Train loss: {}".format(sst_tr_acc/tr_count, cola_tr_acc/tr_count, sst_tr_loss/tr_count, cola_tr_loss/tr_count))
  val_sst_acc, val_sst_loss, val_cola_acc, val_cola_loss = eval_metrics(model, val_loader, criterion)
  #print("SST Val accuracy: {}, CoLA Val accuracy: {}, SST Val loss: {}, CoLA Val loss: {} ".format(val_sst_acc, val_cola_acc, val_sst_loss, val_cola_loss))

100%|██████████| 70/70 [00:06<00:00, 11.51it/s]


SST Train accuracy: 0.5125, CoLA Train accuracy: 0.8767857142857143, SST Train loss: 0.08805537905011858, CoLA Train loss: 0.0577633263277156


100%|██████████| 70/70 [00:03<00:00, 20.43it/s]


SST Train accuracy: 0.5446428571428571, CoLA Train accuracy: 0.8767857142857143, SST Train loss: 0.0868006210241999, CoLA Train loss: 0.05725222038371222


100%|██████████| 70/70 [00:03<00:00, 21.54it/s]


...

SST Train accuracy: 0.5803571428571429, CoLA Train accuracy: 0.8767857142857143, SST Train loss: 0.08520969748497009, CoLA Train loss: 0.055905591802937644


100%|██████████| 70/70 [00:03<00:00, 19.63it/s]


SST Train accuracy: 0.5821428571428572, CoLA Train accuracy: 0.8767857142857143, SST Train loss: 0.08506073914468289, CoLA Train loss: 0.05579671795879092


100%|██████████| 70/70 [00:03<00:00, 20.40it/s]


SST Train accuracy: 0.5839285714285715, CoLA Train accuracy: 0.8767857142857143, SST Train loss: 0.0849229957908392, CoLA Train loss: 0.05570258939904826


In [ ]:
test_sst_acc, test_sst_loss, test_cola_acc, test_cola_loss = eval_metrics(model, test_loader, criterion)
print("SST Test accuracy: {}, CoLA Val accuracy: {}, SST Val loss: {}, CoLA Val loss: {} ".format(test_sst_acc, test_cola_acc, test_sst_loss, test_cola_loss))

SST Test accuracy: 0.8444022770398482, CoLA Val accuracy: 0.6925996204933587, SST Val loss: 0.0016692557837814093, CoLA Val loss: 0.003322326811030507 


In [ ]:
#BERT transfer attacks

sst_adv_dataset = pd.read_csv('bert_adversarial_sst.csv')
cola_adv_dataset = pd.read_csv('bert_adversarial_cola.csv')

adv_set = MultitaskDataset(glove_vocab, tokenizer, sst_adv_dataset['sentences'], sst_adv_dataset['labels'], 
                             cola_adv_dataset['sentences'], cola_adv_dataset['labels'])
adv_loader  = DataLoader(adv_set, shuffle=False, batch_size=8,
                          collate_fn=adv_set.collate_fn)


In [ ]:
adv_sst_acc, adv_sst_loss, adv_cola_acc, adv_cola_loss = eval_metrics(model, adv_loader, criterion)
print("SST ADV accuracy: {}, CoLA ADV accuracy: {}, SST ADV loss: {}, CoLA ADV loss: {} ".format(adv_sst_acc, adv_cola_acc, adv_sst_loss, adv_cola_loss))

SST ADV accuracy: 0.6338983050847458, CoLA ADV accuracy: 0.8491525423728814, SST ADV loss: 0.002810135018080473, CoLA ADV loss: 0.002096907002851367 


In [ ]:
#ROBERTA transfer attack

sst_adv_dataset = pd.read_csv('roberta_adversarial_sst.csv')
cola_adv_dataset = pd.read_csv('roberta_adversarial_cola.csv')

adv_set = MultitaskDataset(glove_vocab, tokenizer, sst_adv_dataset['sentences'], sst_adv_dataset['labels'], 
                             cola_adv_dataset['sentences'], cola_adv_dataset['labels'])
adv_loader  = DataLoader(adv_set, shuffle=False, batch_size=8,
                          collate_fn=adv_set.collate_fn)


In [ ]:
adv_sst_acc, adv_sst_loss, adv_cola_acc, adv_cola_loss = eval_metrics(model, adv_loader, criterion)
print("SST ADV accuracy: {}, CoLA ADV accuracy: {}, SST ADV loss: {}, CoLA ADV loss: {} ".format(adv_sst_acc, adv_cola_acc, adv_sst_loss, adv_cola_loss))

SST ADV accuracy: 0.6035714285714285, CoLA ADV accuracy: 0.8767857142857143, SST ADV loss: 0.002722404198721051, CoLA ADV loss: 0.0020665836054831743 
